### Solution

The solution consists of the following parts: data_handling.py, model construction, cross-validation, training on selected model and test on selected model. I will put them together in this notebook file for explaination. There are another four **.py** files in the directory:
- **model_cnn.py**: code for constructiong the final selected model
- **train_cnn.py**: code for training the final selected model on 80% samples
- **test_cnn.py**: code for testing the final selected model on 20% samples
- **data_handling.py**: some help function to pre-process the data

The **\cv** subdirectory incldudes some models validated in the cross validation step. The final selected and well-trained model is **\cv\cnn_codon1\modelcnn_codon1.hdf5**

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Convolution1D, Input, Masking,MaxPooling1D, Flatten, Dense, Dropout)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from Bio.Seq import Seq
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import (LearningRateScheduler, ModelCheckpoint, Callback, ProgbarLogger)
from tensorflow.keras import backend as K
import os
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Input


#### Data handling part
In this part, I define some methods to transform data into three different form:

(1) One-hot encoding of original DNA sequence (A is [1 0 0 0], C is [0 1 0 0], G is [0 0 1 0], and T is [0 0 0 1])

(2) One-hot encoding of correponding proteins (there are 21 types of proteins in total)

(3) Number sequence (A is 1, C is 2, G is 3 and T is 4)

Note1: these are functions included in the file **data_handling.py**.

Note2: I preprocess all data and store them in the subdirectory **/data**.

-**X_codon.npy**: translated DNA string in string form
**X_codon_one_hot.npy**: tranlated DNA string in one-hot encoding
**X_one_hot_600.npy**: DNA string of first 600 chararcters in one-hot form
**X_one_hot_800.npy**: DNA string of first 800 chararcters in one-hot form
**X_one_hot_1000.npy**: DNA string of first 1000 chararcters in one-hot form

In [7]:
def dna_to_num(s, fixed_length=1000):
    alphabet = {'A':1, 'C':2, 'G':3, 'T':4}
    result = np.zeros(fixed_length)
    for i, char in enumerate(s):
        result[i] = alphabet[char]
    return result

# transforming dna string to one-hot encoding and padding sequence to fixed_length with [0000]
def one_hot_dna(data,fixed_length=1000):
    result = []
    for seq in data:
        result.append(to_categorical(dna_to_num(seq, fixed_length)))
    return np.array(result)

# transforming dna string to number string of '1234' and padding sequence to fixed_length with 0 
def lstm_tokenizer(data, fixed_length):
    tokenizer = Tokenizer(num_words=None, char_level=True)
    tokenizer.fit_on_texts(['ACGT'])
    X = tokenizer.texts_to_sequences(data)
    X = pad_sequences(X, maxlen=fixed_length, padding='post')
    return X

def codon_to_num(s, fixed_length=310):
    codon_dict = np.load('codons_aa.npy', allow_pickle=True).item()
    codon_to_number = {}
    i = 0
    for codon in codon_dict:
        codon_to_number[codon] = i
        i = i + 1
    result = np.zeros(fixed_length)
    for i, char in enumerate(s):
        result[i] = codon_to_number[char]
    return result

# transform dna string to onehot form of protein
def transform_to_codon(data):
    n = len(data)
    translated_result = []
    translated_result_one_hot = []
    for i in range(n):
        # transform dna string to codon
        coding_dna = Seq(xs[i])
        translated = coding_dna.translate()
        translated_result.append(translated)
        # encoding codon string to one-hot form
        translated_result_one_hot.append(to_categorical(codon_to_num(translated)))
    return np.array(translated_result), np.array(translated_result_one_hot)

In [9]:
# demo of data pre-processing

data = np.load("dataset.npy", allow_pickle=True).item()
xs = data["genes"] # [n_sample, arbitrary length string object] 100000 samples
ys = data["resistant"] # [n_sample] (bool)

# transform dna string to one-hot form of fixed length 1000, the reuslt is of the shape(100000, 1000, 4)
X_one_hot_1000 = one_hot_dna(xs, fixed_length=1000)[:, :, 1:5]

# translate string to codon-level and encode in one-hot form, reuslts are of the shape and (100000,) and (100000, 310, 21) res
translated, translated_one_hot = transform_to_codon(xs)

print(translated_one_hot.shape)

(100000, 310, 21)


#### Model construction part

I tried to construct both CNN and LSTM model. But when I tried to check my LSTM model construction (not in cross-validation),I find it takes too long to train on even small size of samplew. So I give up the LSTM model but I will provide code both for  building CNN model and LSTM model by Keras.

I tried to construct different size CNN models and want to choose the best one from cross-validation. The construction code are similar, so I will only provide the code for the model I finally selected. There is a directory "model", all models are stored there in seperate subdirectories. In each corresponding subdirectory of a model, the 'summary-model_name.txt' file contains the summary of model structure.

In [3]:
# Model construction for CNN
seq_len = 310 # Fixed length of a sequence (310 for codon seq, 1000 for some  char-level model,  910 for char-level model)
alph_size = 21 # 21 for codon-level model, 4 for char-level model

#--------------- CNN STRUCTURE ------------------#

#Input one-hot encoded sequence of chars
sequence_one_hot = Input(shape=(seq_len, alph_size), dtype='float32')

# 1st CNN layer without max-pooling
conv1 = Convolution1D(256, 1, activation='relu')(sequence_one_hot)

# 2nd CNN layer with max-pooling
conv2 = Convolution1D(256, 1, activation='relu')(conv1)
pool2 = MaxPooling1D(pool_size=3)(conv2)

# Reshaping to 1D array for further layers
flat = Flatten()(pool2)

# 1st fully connected layer with dropout
dense1 = Dense(1024, activation='relu')(flat)
dropout1 = Dropout(0.5)(dense1)

# 2nd fully connected layer with dropout
dense2 = Dense(1024,activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)

# 3rd fully connected layer with softmax outputs
dense3 = Dense(1, activation='sigmoid')(dropout2)
model = Model(sequence_one_hot, dense3)
model_name ='cnn_codon1'
dir = 'cv\\'+model_name
if os.path.exists(dir):
    model.save(dir+'\\model'+model_name+'.hdf5')
else:
    os.makedirs(dir)
    model.save(dir + '\\model' + model_name + '.hdf5')

print(model.summary())
with open(dir+'\\summary-'+model_name+'.txt', 'w') as fh:
    model.summary(print_fn=lambda x: fh.write(x + '\n'))


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 310, 21)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 310, 256)          5632      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 310, 256)          65792     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 103, 256)          0         
_________________________________________________________________
flatten (Flatten)            (None, 26368)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              27001856  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0     

In [ ]:
# Model construction for LSTM

MAX_NB_WORDS = 5
MAX_SEQUENCE_LENGTH = 200
EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(256,return_sequences=True, kernel_initializer='random_normal'))
model.add(LSTM(256,return_sequences=True,kernel_initializer='random_normal'))
model.add(LSTM(256,kernel_initializer='random_normal'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

model_name ='lstm'
dir = 'model\\'+model_name
if os.path.exists(dir):
    model.save(dir+'\\model'+model_name+'.hdf5')
else:
    os.makedirs(dir)
    model.save(dir + '\\model' + model_name + '.hdf5')

#### Cross-validation part

For now, there are several models:

|Model Name | Max sequence length |model level |Trainable params| Accuracy|
| :-----:|: ----: | :----: |:---------:|:---------:|
|  cnn1  | 1000  | character | 10,822,657| 75.45\%|
|  cnn2  | 1000  | character | 11,806,977|75.18\%|
|  cnn3  | 1000  | character | 87,039,233|76.30\%
|  cnn4_800  | 800  | character |70,849,025 |75.77\%|
|  cnn5_600  | 600  | character | 53,547,521|75.85\%|
|  cnn_codon1  | 310  | codon | 28,123,905|76.60\%|
|  cnn_codon2 | 310  | codon | 27,008,513| 76.00\%|

Because of computation ability, I only used 10000 samples in this part becasue I need to repeat training and validating these samples on all models I built. In this part, I will fisrt keep training parameters as the same for all models and do 5-fold cross-validation on all these models. The initial learing rate is 0.01, the epoch number is 10 and the bathc size is 128. It still takes a long time to finish. There is no huge difference in accuracy of these models. I think this is because all these models have reached the limit of as I only used 10\%  of the original samples. But the training time of the codon-level model is much shorter than that of the character-level (for codon-level model, one epoch takes about 10s but for character-level model of max sequece 1000, one epoch takes more than 1 minute). So I will choose the **cnn_codon2** as the final model.


In [7]:
# Read training data sample
# load data
data = np.load("dataset.npy", allow_pickle=True).item()
# xs = data["genes"] # [n_sample, arbitrary length string object] 100000 samples
ys = data["resistant"] # [n_sample] (bool)
X_codon_one_hot = np.load("data//X_codon_one_hot.npy")
X_one_hot_1000 = np.load("data//X_one_hot_1000.npy")
X_one_hot_800 = np.load("data//X_one_hot_800.npy")
X_one_hot_600 = np.load("data//X_one_hot_600.npy")

X_codon_one_hot_train, X_codon_one_hot_test, y_train, y_test = train_test_split(X_codon_one_hot, ys, test_size=0.2, random_state=42)
X_one_hot_600_train, X_one_hot_600_test, y_train, y_test = train_test_split(X_one_hot_600, ys, test_size=0.2, random_state=42)
X_one_hot_800_train, X_one_hot_800_test, y_train, y_test = train_test_split(X_one_hot_800, ys, test_size=0.2, random_state=42)
X_one_hot_1000_train, X_one_hot_1000_test, y_train, y_test = train_test_split(X_one_hot_1000, ys, test_size=0.2, random_state=42)


In [12]:
# dictory to store model score
model_score= {}

In [8]:
# Self-defined method for step-size decay
def step_size_decay(epoch):
    if epoch > 1 and epoch <= 30 and epoch%3==1:
        global step_size
        step_size = step_size/2
    return step_size

# Self-defined method to print epoch count, loss and step-size (to observe decay) after every epoch
class FlushCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        print('Epoch %s: loss %s' % (epoch, logs.get('loss')))
        print("Step size:",K.eval(optimizer.lr))

In [16]:
# Cross validation with 5-fold on model selection
from sklearn.model_selection import StratifiedKFold
# define 5-fold cross validation test
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

model_dict = {'cnn1': (X_one_hot_1000_train,1000),
              'cnn2': (X_one_hot_1000_train,1000),
              'cnn3': (X_one_hot_1000_train,1000),
              'cnn4_800': (X_one_hot_800_train, 800),
              'cnn5_600': (X_one_hot_600_train, 600),
              'cnn_codon1':(X_codon_one_hot_train, 310),
              'cnn_codon2':(X_codon_one_hot_train,310)}

# stepsize decay
step_size_callback = LearningRateScheduler(step_size_decay)
for model_name in model_dict:
    dir_path = 'cv\\'+model_name
    print(dir_path)
    cvscores = []
    X_train = model_dict[model_name][0]
    for train, val in kfold.split(X_train[0:10000], y_train[0:10000]):
        seq_len = model_dict[model_name][1] # Fixed length of a sequence (310 for codon seq, 1000 for char level seq,  910 for char level)
        max_epochs = 10 
        mini_batch_size = 128
        momentum = 0.9
        step_size = 0.01 # initial step size at the begining of each round
        step_size_callback = LearningRateScheduler(step_size_decay)
        
        all_callbacks = [step_size_callback, FlushCallback(), ProgbarLogger(count_mode='steps')]
        
        model = load_model(dir_path+'\\model'+model_name+'.hdf5', compile=False)
        model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
        
        # Fit the model
        model.fit(X_train[train], y_train[train], batch_size=mini_batch_size, epochs=max_epochs, verbose=2, callbacks=all_callbacks)
        # evaluate the model
        scores = model.evaluate(X_train[val], y_train[val], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    model_score[model_name] = np.mean(cvscores)

cv\cnn4_800
Epoch 1/10
Epoch 0: loss 0.682513952255249
Step size: 0.01
63/63 - 29s - loss: 0.6825 - accuracy: 0.5721 - lr: 0.0100
Epoch 2/10
Epoch 1: loss 0.6614389419555664
Step size: 0.01
63/63 - 30s - loss: 0.6614 - accuracy: 0.6553 - lr: 0.0100
Epoch 3/10
Epoch 2: loss 0.630998969078064
Step size: 0.01
63/63 - 31s - loss: 0.6310 - accuracy: 0.7053 - lr: 0.0100
Epoch 4/10
Epoch 3: loss 0.5880752205848694
Step size: 0.01
63/63 - 33s - loss: 0.5881 - accuracy: 0.7339 - lr: 0.0100
Epoch 5/10
Epoch 4: loss 0.541104257106781
Step size: 0.005
63/63 - 33s - loss: 0.5411 - accuracy: 0.7520 - lr: 0.0050
Epoch 6/10
Epoch 5: loss 0.5124197006225586
Step size: 0.005
63/63 - 34s - loss: 0.5124 - accuracy: 0.7495 - lr: 0.0050
Epoch 7/10
Epoch 6: loss 0.4829428493976593
Step size: 0.005
63/63 - 30s - loss: 0.4829 - accuracy: 0.7530 - lr: 0.0050
Epoch 8/10
Epoch 7: loss 0.4589255452156067
Step size: 0.0025
63/63 - 30s - loss: 0.4589 - accuracy: 0.7526 - lr: 0.0025
Epoch 9/10
Epoch 8: loss 0.4473146

Epoch 7: loss 0.4932791590690613
Step size: 0.0025
63/63 - 24s - loss: 0.4933 - accuracy: 0.7492 - lr: 0.0025
Epoch 9/10
Epoch 8: loss 0.4805707335472107
Step size: 0.0025
63/63 - 24s - loss: 0.4806 - accuracy: 0.7481 - lr: 0.0025
Epoch 10/10
Epoch 9: loss 0.46479636430740356
Step size: 0.0025
63/63 - 25s - loss: 0.4648 - accuracy: 0.7564 - lr: 0.0025
accuracy: 76.35%
Epoch 1/10
Epoch 0: loss 0.6837932467460632
Step size: 0.01
63/63 - 32s - loss: 0.6838 - accuracy: 0.5756 - lr: 0.0100
Epoch 2/10
Epoch 1: loss 0.6637022495269775
Step size: 0.01
63/63 - 27s - loss: 0.6637 - accuracy: 0.6660 - lr: 0.0100
Epoch 3/10
Epoch 2: loss 0.6376554369926453
Step size: 0.01
63/63 - 26s - loss: 0.6377 - accuracy: 0.7029 - lr: 0.0100
Epoch 4/10
Epoch 3: loss 0.6025198101997375
Step size: 0.01
63/63 - 27s - loss: 0.6025 - accuracy: 0.7339 - lr: 0.0100
Epoch 5/10
Epoch 4: loss 0.5660030841827393
Step size: 0.005
63/63 - 26s - loss: 0.5660 - accuracy: 0.7433 - lr: 0.0050
Epoch 6/10
Epoch 5: loss 0.537093

In [17]:
model_score

{'cnn4_800': 75.77000021934509, 'cnn5_600': 75.85000038146973}


Next I will do cross validation on hyperparameter:

- Initial step size(or learning rate): 0.005, 0.01, 0.05, 0.1

- Epoch number: 3, 5, 10, 15

- Batch size: 64, 128, 256

When epoch number is 10 and batch size is 128, the average accuracy rate are 75.46\%, 75.46\%, 79.3\%, 79.08\% for initial step size 0.005, 0.01, 0.05, 0.1 respectively.

When initial step size is 0.05 and batch size is 128, the average accuracy rate are 75.32\%, 75.36\%, 77.48\%, 79.69\% for initial step size epoch number 3,5,10,15 respectively.

When initial step size is 0.05 and epoch number is 15, the average accuracy rate are 77.45\%, 76.34\%, 77.26\% for batch size 64,128,256 respectively.

In [21]:
# Cross validation with 5-fold on hyperparameter selection
# define 5-fold cross validation test
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
model_name = 'cnn_codon1'
step_size_list = [0.005, 0.01, 0.05, 0.1]
score_list = []
# stepsize decay
X_train = X_codon_one_hot_train
step_size_callback = LearningRateScheduler(step_size_decay)
for initial_step_size in step_size_list:
    dir_path = 'cv\\'+model_name
    print(dir_path)
    cvscores = []
    for train, val in kfold.split(X_train[0:10000], y_train[0:10000]):
        seq_len = 310 # Fixed length of a sequence (310 for codon seq, 1000 for char level seq,  910 for char level)
        max_epochs = 10    # Num of epochs training happens
        mini_batch_size = 128
        momentum = 0.9
        step_size = initial_step_size
        step_size_callback = LearningRateScheduler(step_size_decay)
        
        all_callbacks = [step_size_callback, FlushCallback(), ProgbarLogger(count_mode='steps')]
        
        model = load_model(dir_path+'\\model'+model_name+'.hdf5', compile=False)
        model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
        
        # Fit the model
        model.fit(X_train[train], y_train[train], batch_size=mini_batch_size, epochs=max_epochs, verbose=2, callbacks=all_callbacks)
        # evaluate the model
        scores = model.evaluate(X_train[val], y_train[val], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))
        cvscores.append(scores[1] * 100)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    score_list.append(np.mean(cvscores))

cv\cnn_codon1
Epoch 1/10
Epoch 0: loss 0.6751496195793152
Step size: 0.005
63/63 - 12s - loss: 0.6751 - accuracy: 0.6192 - lr: 0.0050
Epoch 2/10
Epoch 1: loss 0.6410863995552063
Step size: 0.005
63/63 - 12s - loss: 0.6411 - accuracy: 0.7253 - lr: 0.0050
Epoch 3/10
Epoch 2: loss 0.6063302755355835
Step size: 0.005
63/63 - 13s - loss: 0.6063 - accuracy: 0.7418 - lr: 0.0050
Epoch 4/10
Epoch 3: loss 0.57016521692276
Step size: 0.005
63/63 - 15s - loss: 0.5702 - accuracy: 0.7530 - lr: 0.0050
Epoch 5/10
Epoch 4: loss 0.5432314872741699
Step size: 0.0025
63/63 - 15s - loss: 0.5432 - accuracy: 0.7552 - lr: 0.0025
Epoch 6/10
Epoch 5: loss 0.5265189409255981
Step size: 0.0025
63/63 - 16s - loss: 0.5265 - accuracy: 0.7539 - lr: 0.0025
Epoch 7/10
Epoch 6: loss 0.5112342238426208
Step size: 0.0025
63/63 - 15s - loss: 0.5112 - accuracy: 0.7517 - lr: 0.0025
Epoch 8/10
Epoch 7: loss 0.4994388222694397
Step size: 0.00125
63/63 - 15s - loss: 0.4994 - accuracy: 0.7545 - lr: 0.0012
Epoch 9/10
Epoch 8: los

Epoch 8/10
Epoch 7: loss 0.39946115016937256
Step size: 0.0025
63/63 - 19s - loss: 0.3995 - accuracy: 0.7601 - lr: 0.0025
Epoch 9/10
Epoch 8: loss 0.3957577347755432
Step size: 0.0025
63/63 - 17s - loss: 0.3958 - accuracy: 0.7577 - lr: 0.0025
Epoch 10/10
Epoch 9: loss 0.3929627537727356
Step size: 0.0025
63/63 - 16s - loss: 0.3930 - accuracy: 0.7515 - lr: 0.0025
accuracy: 75.25%
Epoch 1/10
Epoch 0: loss 0.6581298112869263
Step size: 0.01
63/63 - 16s - loss: 0.6581 - accuracy: 0.6721 - lr: 0.0100
Epoch 2/10
Epoch 1: loss 0.5888853669166565
Step size: 0.01
63/63 - 16s - loss: 0.5889 - accuracy: 0.7514 - lr: 0.0100
Epoch 3/10
Epoch 2: loss 0.5213597416877747
Step size: 0.01
63/63 - 16s - loss: 0.5214 - accuracy: 0.7541 - lr: 0.0100
Epoch 4/10
Epoch 3: loss 0.4638276696205139
Step size: 0.01
63/63 - 16s - loss: 0.4638 - accuracy: 0.7576 - lr: 0.0100
Epoch 5/10
Epoch 4: loss 0.4331476390361786
Step size: 0.005
63/63 - 16s - loss: 0.4331 - accuracy: 0.7556 - lr: 0.0050
Epoch 6/10
Epoch 5: lo

Epoch 5/10
Epoch 4: loss 0.352150022983551
Step size: 0.025
63/63 - 13s - loss: 0.3522 - accuracy: 0.7607 - lr: 0.0250
Epoch 6/10
Epoch 5: loss 0.3499564826488495
Step size: 0.025
63/63 - 14s - loss: 0.3500 - accuracy: 0.7599 - lr: 0.0250
Epoch 7/10
Epoch 6: loss 0.3474500775337219
Step size: 0.025
63/63 - 18s - loss: 0.3475 - accuracy: 0.7628 - lr: 0.0250
Epoch 8/10
Epoch 7: loss 0.34696030616760254
Step size: 0.0125
63/63 - 15s - loss: 0.3470 - accuracy: 0.7669 - lr: 0.0125
Epoch 9/10
Epoch 8: loss 0.34567806124687195
Step size: 0.0125
63/63 - 28s - loss: 0.3457 - accuracy: 0.7634 - lr: 0.0125
Epoch 10/10
Epoch 9: loss 0.34596794843673706
Step size: 0.0125
63/63 - 20s - loss: 0.3460 - accuracy: 0.7640 - lr: 0.0125
accuracy: 74.90%
Epoch 1/10
Epoch 0: loss 0.5467159748077393
Step size: 0.05
63/63 - 18s - loss: 0.5467 - accuracy: 0.7240 - lr: 0.0500
Epoch 2/10
Epoch 1: loss 0.41306060552597046
Step size: 0.05
63/63 - 17s - loss: 0.4131 - accuracy: 0.7487 - lr: 0.0500
Epoch 3/10
Epoch 2

In [24]:
score_list

[75.46000003814697, 75.46000003814697, 79.30000066757202, 79.08000111579895]

In [25]:
# Cross validation with 5-fold on hyperparameter selection
# define 5-fold cross validation test
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
model_name = 'cnn_codon1'
epoch_num_list =[3, 5, 10, 15]
score_list_epoch = []
# stepsize decay
X_train = X_codon_one_hot_train
step_size_callback = LearningRateScheduler(step_size_decay)
for epoch_num in epoch_num_list:
    dir_path = 'cv\\'+model_name
    print(dir_path)
    cvscores = []
    for train, val in kfold.split(X_train[0:10000], y_train[0:10000]):
        seq_len = 310 # Fixed length of a sequence (310 for codon seq, 1000 for char level seq,  910 for char level)
        max_epochs = epoch_num    # Num of epochs training happens
        mini_batch_size = 128
        momentum = 0.9
        step_size = 0.05
        step_size_callback = LearningRateScheduler(step_size_decay)
        
        all_callbacks = [step_size_callback, FlushCallback(), ProgbarLogger(count_mode='steps')]
        
        model = load_model(dir_path+'\\model'+model_name+'.hdf5', compile=False)
        model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
        
        # Fit the model
        model.fit(X_train[train], y_train[train], batch_size=mini_batch_size, epochs=max_epochs, verbose=2, callbacks=all_callbacks)
        # evaluate the model
        scores = model.evaluate(X_train[val], y_train[val], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))
        cvscores.append(scores[1] * 100)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    score_list_epoch.append(np.mean(cvscores))

cv\cnn_codon1
Epoch 1/3
Epoch 0: loss 0.548548698425293
Step size: 0.05
63/63 - 15s - loss: 0.5485 - accuracy: 0.7246 - lr: 0.0500
Epoch 2/3
Epoch 1: loss 0.42181625962257385
Step size: 0.05
63/63 - 14s - loss: 0.4218 - accuracy: 0.7431 - lr: 0.0500
Epoch 3/3
Epoch 2: loss 0.3787739872932434
Step size: 0.05
63/63 - 14s - loss: 0.3788 - accuracy: 0.7539 - lr: 0.0500
accuracy: 75.70%
Epoch 1/3
Epoch 0: loss 0.5472118854522705
Step size: 0.05
63/63 - 13s - loss: 0.5472 - accuracy: 0.7276 - lr: 0.0500
Epoch 2/3
Epoch 1: loss 0.41917961835861206
Step size: 0.05
63/63 - 12s - loss: 0.4192 - accuracy: 0.7516 - lr: 0.0500
Epoch 3/3
Epoch 2: loss 0.38093048334121704
Step size: 0.05
63/63 - 14s - loss: 0.3809 - accuracy: 0.7516 - lr: 0.0500
accuracy: 75.25%
Epoch 1/3
Epoch 0: loss 0.5499602556228638
Step size: 0.05
63/63 - 13s - loss: 0.5500 - accuracy: 0.7285 - lr: 0.0500
Epoch 2/3
Epoch 1: loss 0.4104998707771301
Step size: 0.05
63/63 - 13s - loss: 0.4105 - accuracy: 0.7534 - lr: 0.0500
Epoch 

Epoch 8/10
Epoch 7: loss 0.34820398688316345
Step size: 0.0125
63/63 - 14s - loss: 0.3482 - accuracy: 0.7571 - lr: 0.0125
Epoch 9/10
Epoch 8: loss 0.347734659910202
Step size: 0.0125
63/63 - 14s - loss: 0.3477 - accuracy: 0.7561 - lr: 0.0125
Epoch 10/10
Epoch 9: loss 0.34447041153907776
Step size: 0.0125
63/63 - 13s - loss: 0.3445 - accuracy: 0.7701 - lr: 0.0125
accuracy: 75.10%
Epoch 1/10
Epoch 0: loss 0.5542331337928772
Step size: 0.05
63/63 - 14s - loss: 0.5542 - accuracy: 0.7271 - lr: 0.0500
Epoch 2/10
Epoch 1: loss 0.41094863414764404
Step size: 0.05
63/63 - 13s - loss: 0.4109 - accuracy: 0.7502 - lr: 0.0500
Epoch 3/10
Epoch 2: loss 0.37450313568115234
Step size: 0.05
63/63 - 13s - loss: 0.3745 - accuracy: 0.7575 - lr: 0.0500
Epoch 4/10
Epoch 3: loss 0.3609013259410858
Step size: 0.05
63/63 - 13s - loss: 0.3609 - accuracy: 0.7594 - lr: 0.0500
Epoch 5/10
Epoch 4: loss 0.35254713892936707
Step size: 0.025
63/63 - 13s - loss: 0.3525 - accuracy: 0.7613 - lr: 0.0250
Epoch 6/10
Epoch 5:

Epoch 15/15
Epoch 14: loss 0.3444572389125824
Step size: 0.003125
63/63 - 12s - loss: 0.3445 - accuracy: 0.7646 - lr: 0.0031
accuracy: 79.40%
Epoch 1/15
Epoch 0: loss 0.5462948083877563
Step size: 0.05
63/63 - 13s - loss: 0.5463 - accuracy: 0.7312 - lr: 0.0500
Epoch 2/15
Epoch 1: loss 0.40817001461982727
Step size: 0.05
63/63 - 13s - loss: 0.4082 - accuracy: 0.7559 - lr: 0.0500
Epoch 3/15
Epoch 2: loss 0.3798178434371948
Step size: 0.05
63/63 - 13s - loss: 0.3798 - accuracy: 0.7530 - lr: 0.0500
Epoch 4/15
Epoch 3: loss 0.36443284153938293
Step size: 0.05
63/63 - 21s - loss: 0.3644 - accuracy: 0.7567 - lr: 0.0500
Epoch 5/15
Epoch 4: loss 0.35288020968437195
Step size: 0.025
63/63 - 15s - loss: 0.3529 - accuracy: 0.7611 - lr: 0.0250
Epoch 6/15
Epoch 5: loss 0.3506484925746918
Step size: 0.025
63/63 - 13s - loss: 0.3506 - accuracy: 0.7595 - lr: 0.0250
Epoch 7/15
Epoch 6: loss 0.34966838359832764
Step size: 0.025
63/63 - 13s - loss: 0.3497 - accuracy: 0.7605 - lr: 0.0250
Epoch 8/15
Epoch 7

In [26]:
score_list_epoch

[75.32000064849854, 75.35999894142151, 77.48000025749207, 79.69000101089478]

In [27]:
# Cross validation with 5-fold on hyperparameter selection
# define 5-fold cross validation test
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
model_name = 'cnn_codon1'
batch_size_list = [64,128,256]
score_list_batch_size = []
# stepsize decay
X_train = X_codon_one_hot_train
step_size_callback = LearningRateScheduler(step_size_decay)
for batch_size in batch_size_list:
    dir_path = 'cv\\'+model_name
    print(dir_path)
    cvscores = []
    for train, val in kfold.split(X_train[0:10000], y_train[0:10000]):
        seq_len = 310 # Fixed length of a sequence (310 for codon seq, 1000 for char level seq,  910 for char level)
        max_epochs = 15    # Num of epochs training happens
        mini_batch_size = batch_size
        momentum = 0.9
        step_size = 0.05
        step_size_callback = LearningRateScheduler(step_size_decay)
        
        all_callbacks = [step_size_callback, FlushCallback(), ProgbarLogger(count_mode='steps')]
        
        model = load_model(dir_path+'\\model'+model_name+'.hdf5', compile=False)
        model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
        
        # Fit the model
        model.fit(X_train[train], y_train[train], batch_size=mini_batch_size, epochs=max_epochs, verbose=2, callbacks=all_callbacks)
        # evaluate the model
        scores = model.evaluate(X_train[val], y_train[val], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))
        cvscores.append(scores[1] * 100)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    score_list_batch_size.append(np.mean(cvscores))

cv\cnn_codon1
Epoch 1/15
Epoch 0: loss 0.4888020157814026
Step size: 0.05
125/125 - 15s - loss: 0.4888 - accuracy: 0.7371 - lr: 0.0500
Epoch 2/15
Epoch 1: loss 0.37657392024993896
Step size: 0.05
125/125 - 16s - loss: 0.3766 - accuracy: 0.7576 - lr: 0.0500
Epoch 3/15
Epoch 2: loss 0.3592732846736908
Step size: 0.05
125/125 - 20s - loss: 0.3593 - accuracy: 0.7523 - lr: 0.0500
Epoch 4/15
Epoch 3: loss 0.35499218106269836
Step size: 0.05
125/125 - 20s - loss: 0.3550 - accuracy: 0.7492 - lr: 0.0500
Epoch 5/15
Epoch 4: loss 0.34838971495628357
Step size: 0.025
125/125 - 19s - loss: 0.3484 - accuracy: 0.7588 - lr: 0.0250
Epoch 6/15
Epoch 5: loss 0.34831127524375916
Step size: 0.025
125/125 - 19s - loss: 0.3483 - accuracy: 0.7540 - lr: 0.0250
Epoch 7/15
Epoch 6: loss 0.3473849594593048
Step size: 0.025
125/125 - 15s - loss: 0.3474 - accuracy: 0.7581 - lr: 0.0250
Epoch 8/15
Epoch 7: loss 0.3458269238471985
Step size: 0.0125
125/125 - 15s - loss: 0.3458 - accuracy: 0.7619 - lr: 0.0125
Epoch 9/1

Epoch 7/15
Epoch 6: loss 0.3444690406322479
Step size: 0.025
125/125 - 20s - loss: 0.3445 - accuracy: 0.7560 - lr: 0.0250
Epoch 8/15
Epoch 7: loss 0.34220439195632935
Step size: 0.0125
125/125 - 20s - loss: 0.3422 - accuracy: 0.7651 - lr: 0.0125
Epoch 9/15
Epoch 8: loss 0.342184841632843
Step size: 0.0125
125/125 - 21s - loss: 0.3422 - accuracy: 0.7656 - lr: 0.0125
Epoch 10/15
Epoch 9: loss 0.34243127703666687
Step size: 0.0125
125/125 - 21s - loss: 0.3424 - accuracy: 0.7641 - lr: 0.0125
Epoch 11/15
Epoch 10: loss 0.3402852714061737
Step size: 0.00625
125/125 - 21s - loss: 0.3403 - accuracy: 0.7695 - lr: 0.0063
Epoch 12/15
Epoch 11: loss 0.3405463695526123
Step size: 0.00625
125/125 - 20s - loss: 0.3405 - accuracy: 0.7666 - lr: 0.0063
Epoch 13/15
Epoch 12: loss 0.34088611602783203
Step size: 0.00625
125/125 - 20s - loss: 0.3409 - accuracy: 0.7650 - lr: 0.0063
Epoch 14/15
Epoch 13: loss 0.3390892744064331
Step size: 0.003125
125/125 - 20s - loss: 0.3391 - accuracy: 0.7715 - lr: 0.0031
E

Epoch 14/15
Epoch 13: loss 0.34438347816467285
Step size: 0.003125
63/63 - 13s - loss: 0.3444 - accuracy: 0.7581 - lr: 0.0031
Epoch 15/15
Epoch 14: loss 0.3416717052459717
Step size: 0.003125
63/63 - 30s - loss: 0.3417 - accuracy: 0.7686 - lr: 0.0031
accuracy: 74.95%
Epoch 1/15
Epoch 0: loss 0.5495885610580444
Step size: 0.05
63/63 - 14s - loss: 0.5496 - accuracy: 0.7372 - lr: 0.0500
Epoch 2/15
Epoch 1: loss 0.41143012046813965
Step size: 0.05
63/63 - 13s - loss: 0.4114 - accuracy: 0.7554 - lr: 0.0500
Epoch 3/15
Epoch 2: loss 0.3735223114490509
Step size: 0.05
63/63 - 13s - loss: 0.3735 - accuracy: 0.7595 - lr: 0.0500
Epoch 4/15
Epoch 3: loss 0.36088407039642334
Step size: 0.05
63/63 - 22s - loss: 0.3609 - accuracy: 0.7560 - lr: 0.0500
Epoch 5/15
Epoch 4: loss 0.3506428599357605
Step size: 0.025
63/63 - 23s - loss: 0.3506 - accuracy: 0.7616 - lr: 0.0250
Epoch 6/15
Epoch 5: loss 0.3489919900894165
Step size: 0.025
63/63 - 13s - loss: 0.3490 - accuracy: 0.7641 - lr: 0.0250
Epoch 7/15
Epo

Epoch 6/15
Epoch 5: loss 0.3685418665409088
Step size: 0.025
32/32 - 12s - loss: 0.3685 - accuracy: 0.7548 - lr: 0.0250
Epoch 7/15
Epoch 6: loss 0.36199209094047546
Step size: 0.025
32/32 - 12s - loss: 0.3620 - accuracy: 0.7581 - lr: 0.0250
Epoch 8/15
Epoch 7: loss 0.3593674600124359
Step size: 0.0125
32/32 - 16s - loss: 0.3594 - accuracy: 0.7589 - lr: 0.0125
Epoch 9/15
Epoch 8: loss 0.3567412197589874
Step size: 0.0125
32/32 - 14s - loss: 0.3567 - accuracy: 0.7585 - lr: 0.0125
Epoch 10/15
Epoch 9: loss 0.35474738478660583
Step size: 0.0125
32/32 - 12s - loss: 0.3547 - accuracy: 0.7632 - lr: 0.0125
Epoch 11/15
Epoch 10: loss 0.35404178500175476
Step size: 0.00625
32/32 - 12s - loss: 0.3540 - accuracy: 0.7590 - lr: 0.0063
Epoch 12/15
Epoch 11: loss 0.35373127460479736
Step size: 0.00625
32/32 - 12s - loss: 0.3537 - accuracy: 0.7567 - lr: 0.0063
Epoch 13/15
Epoch 12: loss 0.3519732654094696
Step size: 0.00625
32/32 - 21s - loss: 0.3520 - accuracy: 0.7624 - lr: 0.0063
Epoch 14/15
Epoch 13

In [28]:
score_list_batch_size

[77.45000004768372, 76.33999824523926, 77.260000705719]

#### Training Part

I train the **cnn_codon1** model based on the selected hyperparameters with all training samples(80000).I ser the initial step size as 0.05. I increase the epoch number to 20 because there are more training sample now. I choose the batch size as 128 because the larger data size although this is not the best from cross-validation. 

In [29]:
seq_len = 310
max_epochs = 20     # Num of epochs training happens
mini_batch_size = 128
step_size = 0.05
model_name = 'cnn_codon1'
dir = 'cv\\'+model_name

data = np.load("dataset.npy", allow_pickle=True).item()
ys = data["resistant"]  # [n_sample] (bool)
X_train = X_codon_one_hot_train

step_size_callback = LearningRateScheduler(step_size_decay)

# Callbacks to save and retreive the best weight configurations found during training phase
all_callbacks = [step_size_callback, FlushCallback(),
                           ModelCheckpoint(dir+'\\model'+model_name+'.hdf5',
                                           save_best_only=True, verbose=1),
                           ProgbarLogger(count_mode='steps')]

#--------------- TRAINING ------------------#
model = load_model(dir+'\\model'+model_name+'.hdf5', compile=False)
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

hist = model.fit(X_train, y_train, batch_size=mini_batch_size, epochs=max_epochs,
                 verbose=2, validation_split=0.2, callbacks=all_callbacks)
print(hist.history, file=open(dir+'\\history-'+model_name+'.txt', 'a'))
print(hist.history)
#  Write training history into csv file
hist_df = pd.DataFrame(hist.history)
if os.path.exists(dir+'\\'+model_name+'.csv'):
    hist_df.to_csv(dir + '\\' + model_name + '.csv', mode='a', header=False, index=False)
else:
    hist_df.to_csv(dir + '\\' + model_name + '.csv', mode='a', header=True, index=False)


Epoch 1/20
Epoch 0: loss 0.3950274586677551
Step size: 0.05

Epoch 00001: val_loss improved from inf to 0.34697, saving model to cv\cnn_codon1\modelcnn_codon1.hdf5
500/500 - 113s - loss: 0.3950 - accuracy: 0.7481 - val_loss: 0.3470 - val_accuracy: 0.7535 - lr: 0.0500
Epoch 2/20
Epoch 1: loss 0.3516567647457123
Step size: 0.05

Epoch 00002: val_loss improved from 0.34697 to 0.34490, saving model to cv\cnn_codon1\modelcnn_codon1.hdf5
500/500 - 125s - loss: 0.3517 - accuracy: 0.7527 - val_loss: 0.3449 - val_accuracy: 0.7535 - lr: 0.0500
Epoch 3/20
Epoch 2: loss 0.34905344247817993
Step size: 0.05

Epoch 00003: val_loss improved from 0.34490 to 0.34353, saving model to cv\cnn_codon1\modelcnn_codon1.hdf5
500/500 - 125s - loss: 0.3491 - accuracy: 0.7573 - val_loss: 0.3435 - val_accuracy: 0.7535 - lr: 0.0500
Epoch 4/20
Epoch 3: loss 0.34737634658813477
Step size: 0.05

Epoch 00004: val_loss did not improve from 0.34353
500/500 - 143s - loss: 0.3474 - accuracy: 0.7645 - val_loss: 0.3463 - val_

#### Test Part

The final accuracy on the test set of 200000 samples is 99.52\%

In [30]:
model_name = 'cnn_codon1'
dir = 'cv\\'+model_name

model = load_model(dir+'\\model'+model_name+'.hdf5', compile=False)
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

X_test = X_codon_one_hot_test
y_pred = model.predict(X_test)
model.evaluate(X_test, y_test)

625/625 [==============================] - 13s 21ms/step - loss: 0.0203 - accuracy: 0.99522s - los


[0.020292269065976143, 0.995199978351593]

#### Challenge

The biggest challenge is the computation ability. If I had a GPU, I can do cross validation on more samples and tried more model construction such as changing the number og units in each layer, changing the drop rate and shrinking the input sequence length. And I can try different optimization methods.

Another challenge is about the sequence length. In this problem, I preprocessed all data and I found that the maximum length is below 1000, which means it is possible to keep all data. But I am not sure if this will always hold in other dataset. On the other hand, I know "ATG" is the start codon based on my biology knowledge. Most(99.5\%) of the DNA strings start with "ATG", but for those which start with other codon, I am not sure if it is fine to translted them into codon string in the defalut way that the method "coding_dna.translate()" dose. 